In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import glob
import math
import random

# Direct Linear Transform(DLT)

In [2]:
calib_img = cv2.imread('../input_data/calib-object.jpg')
%matplotlib qt 
plt.imshow(calib_img)

In [5]:
"""X represents set of world coordinates, x represents set of image coordinates"""
def dlt(X, x):
    M = np.zeros((2 * len(x), 12))
    for i in range(len(x)):
        M[2 * i][0] = -X[i][0]
        M[2 * i][1] = -X[i][1]
        M[2 * i][2] = -X[i][2]
        M[2 * i][3] = -1
        M[2 * i][4] = 0
        M[2 * i][5] = 0
        M[2 * i][6] = 0
        M[2 * i][7] = 0
        M[2 * i][8] = x[i][0] * X[i][0]
        M[2 * i][9] = x[i][0] * X[i][1]
        M[2 * i][10] = x[i][0] * X[i][2]
        M[2 * i][11] = x[i][0]

        M[2 * i + 1][0] = 0
        M[2 * i + 1][1] = 0
        M[2 * i + 1][2] = 0
        M[2 * i + 1][3] = 0
        M[2 * i + 1][4] = -X[i][0]
        M[2 * i + 1][5] = -X[i][1]
        M[2 * i + 1][6] = -X[i][2]
        M[2 * i + 1][7] = -1
        M[2 * i + 1][8] = x[i][1] * X[i][0]
        M[2 * i + 1][9] = x[i][1] * X[i][1]
        M[2 * i + 1][10] = x[i][1] * X[i][2]
        M[2 * i + 1][11] = x[i][1]
    u, s, v = np.linalg.svd(M)
    P = v[11].reshape((3, 4))
    KR = P[:, 0:3]
    minus_KRC = P[:, 3]
    C = np.dot(-np.linalg.inv(KR), minus_KRC)
    ################ Decide which to keep ##################################
    """ Option 1 """
    R_T, K_cap_inv = np.linalg.qr(np.linalg.inv(KR))
    R = R_T.T
    K_cap = np.linalg.inv(K_cap_inv)
    K = K_cap / K_cap[2, 2]
    """ Option 2 """
#     R, K_cap = np.linalg.qr(KR)
#     K = K_cap / K_cap[2, 2]
    ##################################################
    return P, C, K, R

In [30]:
x = []
X = []
def add_point(px_coord, world_coord):
    x.append(px_coord)
    X.append(world_coord)


add_point([1409, 1606], [0, 0, 28])
add_point([1134, 1640], [0, 0, 84])
add_point([1662, 1610], [28, 0, 0])
add_point([1932, 1644], [84, 0, 0])
add_point([1666, 1784], [28, 28, 0])
add_point([2412, 1917], [168, 28, 0])

add_point([1545, 1591], [0, 0, 0])
add_point([1789, 1983], [56, 56, 0])
add_point([2245, 1685], [140, 0, 0])
add_point([1412, 1781], [0, 28, 28])
add_point([1280, 1977], [0, 56, 56])
add_point([1794, 1627], [56, 0, 0])
add_point([1279, 1623], [0, 0, 56])


add_point([1658, 2903], [28, 224, 0])
add_point([1777, 2960], [56, 224, 0])
add_point([1905, 3021], [84, 224, 0])
add_point([1161, 2855], [0, 196, 84])
add_point([857, 2802], [0, 168, 140])
add_point([1545, 2399], [0, 140, 0])
add_point([2599, 1949], [196, 28, 0])
# add_point([1923, 2012], [84, 56, 0])

P, C, K, R = dlt(X, x)
print("Projection Matrix : \n", P, "\n")
print("Projection Center : \n", C, "\n")
print("Camera Matrix : \n", K, "\n")
print("Rotation Matrix : \n", R, "\n")

Projection Matrix : 
 [[ 1.08705782e-03  2.54977186e-04 -2.78833812e-03  6.95493439e-01]
 [-5.87153430e-04  2.98849545e-03 -5.84591121e-04  7.18519335e-01]
 [-5.26797446e-07  1.61152247e-07 -5.12468957e-07  4.51147055e-04]] 

Projection Center : 
 [433.21779347 -74.82221824 411.48104003] 

Camera Matrix : 
 [[3.66376509e+03 1.21994004e+01 1.58514696e+03]
 [0.00000000e+00 3.64393861e+03 1.92630130e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] 

Rotation Matrix : 
 [[-0.69674911  0.00342397  0.71730673]
 [-0.15596734 -0.97678745 -0.14683484]
 [ 0.70015346 -0.21418347  0.68110982]] 



# Zhangs Method

In [25]:
# Visualizing the corners
img = cv2.imread('../input_data/IMG_5457.JPG')
ret, corners = cv2.findChessboardCorners(img,(6, 8))
corners = corners.reshape(-1,2)

img_copy = img.copy()
cv2.drawChessboardCorners(img_copy, (6, 8), corners, ret)
plt.imshow(img_copy)
print(corners[0])
print(corners[6])

[1136.0684 2769.522 ]
[1528.  2792.5]


In [26]:
x,y = np.meshgrid(range(6),range(8))
world_points = np.hstack((x.reshape(48, 1),y.reshape(48, 1),np.zeros((48, 1)))).astype(np.float32)

In [31]:
_3d = []
_2d = []
for name in range(5456, 5471):
    img = cv2.imread('../input_data/IMG_' + str(name) + '.JPG')
    ret, corners = cv2.findChessboardCorners(img,(6, 8))
    if ret is True:
        _2d.append(corners)
        _3d.append(world_points)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(_3d, _2d, (img.shape[1],img.shape[0]), None, None)
print("Camera Matrix : \n", mtx, "\n")
print("Reprojection error : ", ret, "\n")

Camera Matrix : 
 [[1.36634814e+04 0.00000000e+00 3.33651179e+03]
 [0.00000000e+00 1.36813888e+04 1.49657922e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] 

Reprojection error :  2.4810222497142798 

